In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

import pandas as pd
from tqdm import tqdm

parent = os.path.abspath(os.path.join(".."))
sys.path.append(parent)

import youtube.youtube_api as youtube_api

Celeb list

In [14]:
api_keys = []
youtuber = youtube_api.YouTubeAPI(api_keys.pop())

In [15]:
csv_path = "../urls/vox2_meta.csv"
celeb_df = pd.read_csv(csv_path)
celeb_list = list(celeb_df["Name"].values)

In [16]:
csv_path = "../urls/yt-@celebs4k-search.csv"
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    df = pd.DataFrame(columns=["url", "video_id", "celeb"])

In [18]:
for celeb in tqdm(celeb_list, desc="searching"):
    if celeb in list(set(list(df["celeb"].values))):
        continue

    search_term = celeb + " interview 4k"
    video_ids = youtuber.search_youtube(search_term)

    if len(video_ids) == 0:
        if len(api_keys) == 0:
            break
        else:
            print("changing api-key...")
            youtuber = youtube_api.YouTubeAPI(api_keys.pop())
            video_ids = youtuber.search_youtube(search_term)

    for video_id in video_ids:
        video_info = {
            "url": "https://www.youtube.com/watch?v=" + video_id,
            "video_id": video_id,
            "celeb": celeb,
        }
        df = pd.concat([df, pd.DataFrame([video_info])], ignore_index=True)

df = df.drop_duplicates(subset="video_id", keep=False)
df.to_csv(csv_path, index=False)

searching:   1%|▏         | 80/6114 [00:00<00:15, 396.70it/s]

searching:   3%|▎         | 162/6114 [00:00<00:14, 399.91it/s]

changing api-key...


searching:  55%|█████▌    | 3374/6114 [00:09<00:06, 398.64it/s]

changing api-key...


searching:  83%|████████▎ | 5082/6114 [00:14<00:04, 227.41it/s]

changing api-key...


searching: 100%|██████████| 6114/6114 [00:17<00:00, 348.75it/s]


Split

In [19]:
def split_dataframe_to_chunks(df, k, shuffle=True):

    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    num_chunks = len(df) // k + (1 if len(df) % k else 0)
    chunks = [df[i * k : (i + 1) * k] for i in range(num_chunks)]

    return chunks


csv_path = "../urls/yt-@celebs4k-search.csv"
ref_csv_path = "../urls/yt-@celebs4k.csv"

df = pd.read_csv(csv_path)
ref_df = pd.read_csv(ref_csv_path)

values_to_remove = ref_df["video_id"].unique()
df = df[~df["video_id"].isin(values_to_remove)]

dir_name = os.path.join("../urls/", os.path.basename(csv_path).replace(".csv", ""))
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

chunks = split_dataframe_to_chunks(df, k=5000)
for i, chunk in enumerate(chunks):
    chunk.to_csv(
        os.path.join(
            dir_name, os.path.basename(csv_path).replace(".csv", "-{}.csv".format(str(i).zfill(2)))
        ),
        index=False,
    )